And here we start experimenting with Linear Regression

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
house = pd.read_csv('2_all_houses.txt')

In [3]:
house

,Number of rooms,Square footage,Price,Year built,Type,Address,City
0,3,61,284500,1983,Appartement,Nes 96,Pijnacker
1,4,104,375000,1983,Woonhuis,Roggeakker 23,Pijnacker
2,4,277,1075000,2006,Woonhuis,Katwijkerlaan 119,Pijnacker
3,5,101,450000,2002,Woonhuis,Rodenrijseweg 547,Berkel en Rodenrijs
4,6,140,600000,2002,Woonhuis,Gouden Harplaan 27,Berkel en Rodenrijs
...,...,...,...,...,...,...,...
368,9,280,1695000,1985,Woonhuis,Ringvaartweg 211,Rotterdam
369,4,111,730000,1985,Appartement,Baan 54 S,Rotterdam
370,3,101,725000,1985,Appartement,Baan 50 M,Rotterdam
371,4,140,969000,1985,Appartement,Baan 56 C,Rotterdam
